## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_Vacation.csv")

#world_weather_analysis/Vacation_Search/WeatherPy_Vacation.csv

vacation_df.head(15)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Yarada,IN,77.32,mist,17.6500,83.2667,Novotel Visakhapatnam Varun Beach
1,6,Carnarvon,AU,80.67,clear sky,-24.8667,113.6333,Hospitality Carnarvon
2,7,Rikitea,PF,76.89,overcast clouds,-23.1203,-134.9692,People ThankYou
3,11,Butaritari,KI,81.25,light rain,3.0707,172.7902,Isles Sunset Lodge
4,18,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Little Polynesian Resort
5,23,Dingle,PH,81.27,broken clouds,10.9995,122.6711,Hotel Del Rio
6,26,Vaini,TO,86.16,broken clouds,-21.2000,-175.2000,Little Italy Hotel
7,40,Kapaa,US,78.78,heavy intensity rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,44,Bathsheba,BB,81.21,scattered clouds,13.2167,-59.5167,Hilton Barbados Resort
9,49,Quatre Cocos,MU,77.11,clear sky,-20.2078,57.7625,The Ravenala Attitude Hotel


### Reduce the DF for testing

In [3]:
#Reduce the DF for testing
# vacation_df = vacation_df.head(10)
# vacation_df.count()

In [4]:
# 2. Using the template add the city name, the country code,
# the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [5]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[['Lat', 'Lng']]
max_temp = vacation_df['Max Temp']

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# From the map above pick 4 cities and create a vacation itinerary route to
#travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

start_city = 'Ico'
end_city = 'Ico'
stop01 = 'Novo Oriente'
stop02 = 'Nova Russas'
stop03 = 'Senador Pompeu'

vacation_start = vacation_df.loc[vacation_df['City'] == start_city]
vacation_end = vacation_df.loc[vacation_df['City'] == end_city]
vacation_stop1 = vacation_df.loc[vacation_df['City'] == stop01]
vacation_stop2 = vacation_df.loc[vacation_df['City'] == stop02] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == stop03] 

In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using
# the to_numpy function and list indexing.
# start = (-6.4011, -38.8622)
# end = (-6.4011, -38.8622)
# stop1 = (-5.5344, -40.7742)
# stop2 = (-4.7067, -40.5631)
# stop3 = (-5.5881, -39.3717)

travel_start = vacation_start['Lat'].to_numpy()[0], vacation_start['Lng'].to_numpy()[0]
travel_end = vacation_end['Lat'].to_numpy()[0], vacation_end['Lng'].to_numpy()[0]
travel_stop1 = vacation_stop1['Lat'].to_numpy()[0], vacation_stop1['Lng'].to_numpy()[0]
travel_stop2 = vacation_stop2['Lat'].to_numpy()[0], vacation_stop2['Lng'].to_numpy()[0]
travel_stop3 = vacation_stop3['Lat'].to_numpy()[0], vacation_stop3['Lng'].to_numpy()[0]

In [8]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints.
#The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
travel_path = gmaps.directions_layer(
        travel_start, travel_end, waypoints=[travel_stop1, travel_stop2, travel_stop3],
         travel_mode='DRIVING')
fig.add_layer(travel_path)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1,
                          vacation_stop2, vacation_stop3],
                         ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1870,Ico,BR,83.34,broken clouds,-6.4011,-38.8622,Lanchonete e Pousada Alameda
1,1500,Novo Oriente,BR,80.91,broken clouds,-5.5344,-40.7742,Koncord Motel
2,2249,Nova Russas,BR,82.02,broken clouds,-4.7067,-40.5631,palace motel
3,2229,Senador Pompeu,BR,80.73,broken clouds,-5.5881,-39.3717,Mombaça Premium Hotel


In [10]:
# 9 Using the template add city name, the country code, the weather description
# and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
itinerary_locations = itinerary_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map.

fig1 = gmaps.figure()
itinerary_path = gmaps.directions_layer(travel_start, travel_end,
                                     waypoints=[travel_stop1, travel_stop2, travel_stop3], show_markers=False,
                                     travel_mode='DRIVING')
marker_layer = gmaps.marker_layer(itinerary_locations, info_box_content=vacation_info, display_info_box=True)
fig1.add_layer(itinerary_path)
fig1.add_layer(marker_layer)


# 11b. Display the figure
fig1

Figure(layout=FigureLayout(height='420px'))